In [1]:
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Cargar un modelo preentrenado, por ejemplo, ResNet50
model = torchvision.models.resnet50(pretrained=True)

# Asegúrate de que el modelo esté en la misma ubicación que los datos (en la GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Ahora, activar los gradientes solo para las capas que quieres entrenar
# Ejemplo: si estamos usando LoRA, activamos los gradientes en las capas modificadas
for param in model.layer4.parameters():  # Asumiendo que layer4 es la capa modificada por LoRA
    param.requires_grad = True

# Imprimir los parámetros que están siendo entrenados
print("Parámetros entrenables:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f" - {name}")

# Definir las transformaciones
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización similar a la de ImageNet
])

# Cargar el conjunto de datos
dataset = datasets.CIFAR10(root='Pruebalora2', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Definir el optimizador (aquí estamos entrenando LoRA)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# Definir la función de pérdida
criterion = nn.CrossEntropyLoss()

# Ajuste fino (Fine-Tuning)
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0
    model.train()  # Establecer el modelo en modo entrenamiento
    for batch_idx, (images, labels) in enumerate(train_loader):
        
        # Enviar imágenes y etiquetas al dispositivo (si usas GPU)
        images, labels = images.to(device), labels.to(device)

        # Adelante (Forward)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Retropropagación (Backward)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()

        # Imprimir la pérdida de cada lote
        if (batch_idx + 1) % 10 == 0:  # Imprimir cada 10 lotes
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Imprimir la pérdida promedio de la época
    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {epoch_loss/len(train_loader):.4f}")


C:\Users\lucia\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lucia\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Parámetros entrenables:
 - layer4.0.conv1.weight
 - layer4.0.bn1.weight
 - layer4.0.bn1.bias
 - layer4.0.conv2.weight
 - layer4.0.bn2.weight
 - layer4.0.bn2.bias
 - layer4.0.conv3.weight
 - layer4.0.bn3.weight
 - layer4.0.bn3.bias
 - layer4.0.downsample.0.weight
 - layer4.0.downsample.1.weight
 - layer4.0.downsample.1.bias
 - layer4.1.conv1.weight
 - layer4.1.bn1.weight
 - layer4.1.bn1.bias
 - layer4.1.conv2.weight
 - layer4.1.bn2.weight
 - layer4.1.bn2.bias
 - layer4.1.conv3.weight
 - layer4.1.bn3.weight
 - layer4.1.bn3.bias
 - layer4.2.conv1.weight
 - layer4.2.bn1.weight
 - layer4.2.bn1.bias
 - layer4.2.conv2.weight
 - layer4.2.bn2.weight
 - layer4.2.bn2.bias
 - layer4.2.conv3.weight
 - layer4.2.bn3.weight
 - layer4.2.bn3.bias
Epoch [1/10], Batch [10/1563], Loss: 5.6221
Epoch [1/10], Batch [20/1563], Loss: 3.7662
Epoch [1/10], Batch [30/1563], Loss: 3.4397
Epoch [1/10], Batch [40/1563], Loss: 3.0849
Epoch [1/10], Batch [50/1563], Loss: 1.6579
Epoch [1/10], Batch [60/1563], Loss: 2.02

In [3]:
# Guardar el modelo entrenado
model_path = 'modelo_entrenado.pth'  # Puedes cambiar el nombre de archivo y la ruta
torch.save(model.state_dict(), model_path)
print(f"Modelo guardado en {model_path}")

Modelo guardado en modelo_entrenado.pth


In [32]:
import torch
print(torch.cuda.is_available())  # Debería devolver True si CUDA está disponible


True


In [9]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip list

In [ ]:
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"Nombre de la GPU: {t63orch.cuda.get_device_name(0)}")
